In [1]:
from pynwb import NWBHDF5IO
import numpy as np

import numpy as np
from pynwb import NWBHDF5IO
import pandas as pd
import numpy as np
import pyarrow.parquet as pq
from pyspark.sql import SparkSession
from pyspark.sql import functions as F
import pyarrow as pa
import pyarrow.parquet as pq


In [2]:
import scipy.io
import pandas as pd
import numpy as np
from tqdm import tqdm
import pyarrow as pa
import pyarrow.parquet as pq
import mat73

In [3]:
import h5py

file = h5py.File('Center_out/Churchland2021/sub-Nitschke_ses-20090920_behavior+ecephys.nwb')

io = NWBHDF5IO(file=file, mode='r', load_namespaces=True)

/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/hdmf/utils.py:668: UserWarning: Ignoring cached namespace 'hdmf-common' version 1.3.0 because version 1.8.0 is already loaded.
  return func(args[0], **pargs)
/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/hdmf/utils.py:668: UserWarning: Ignoring cached namespace 'core' version 2.2.5 because version 2.6.0-alpha is already loaded.
  return func(args[0], **pargs)


In [4]:
nwb = io.read()
nwb

,x,y,z,imp,location,filtering,group,group_name,offset,gain
id,,,,,,,,,,
1,NaN,NaN,NaN,-1.0,"Pre-Motor Cortex, dorsal",1000Hz,"1 pynwb.ecephys.ElectrodeGroup at 0x4999977872\nFields:\n description: array corresponding to device implanted at PMd\n device: Utah Array(PMd) pynwb.device.Device at 0x5000072848\nFields:\n description: 96 channel utah array\n manufacturer: BlackRock Microsystems\n\n location: Caudal, dorsal Pre-motor cortex, Left hemisphere\n",1,0.0,1.0
2,NaN,NaN,NaN,-1.0,"Pre-Motor Cortex, dorsal",1000Hz,"1 pynwb.ecephys.ElectrodeGroup at 0x4999977872\nFields:\n description: array corresponding to device implanted at PMd\n device: Utah Array(PMd) pynwb.device.Device at 0x5000072848\nFields:\n description: 96 channel utah array\n manufacturer: BlackRock Microsystems\n\n location: Caudal, dorsal Pre-motor cortex, Left hemisphere\n",1,0.0,1.0
3,NaN,NaN,NaN,-1.0,"Pre-Motor Cortex, dorsal",1000Hz,"1 pynwb.ecephys.ElectrodeGroup at 0x4999977872\nFields:\n description: array corresponding to device implanted at PMd\n device: Utah Array(PMd) pynwb.device.Device at 0x5000072848\nFields:\n description: 96 channel utah array\n manufacturer: BlackRock Microsystems\n\n location: Caudal, dorsal Pre-motor cortex, Left hemisphere\n",1,0.0,1.0
4,NaN,NaN,NaN,-1.0,"Pre-Motor Cortex, dorsal",1000Hz,"1 pynwb.ecephys.ElectrodeGroup at 0x4999977872\nFields:\n description: array corresponding to device implanted at PMd\n device: Utah Array(PMd) pynwb.device.Device at 0x5000072848\nFields:\n description: 96 channel utah array\n manufacturer: BlackRock Microsystems\n\n location: Caudal, dorsal Pre-motor cortex, Left hemisphere\n",1,0.0,1.0
,x,y,z,imp,location,filtering,group,group_name,offset,gain
id,,,,,,,,,,
1,NaN,NaN,NaN,-1.0,"Pre-Motor Cortex, dorsal",1000Hz,"1 pynwb.ecephys.ElectrodeGroup at 0x4999977872\nFields:\n description: array corresponding to device implanted at PMd\n device: Utah Array(PMd) pynwb.device.Device at 0x5000072848\nFields:\n description: 96 channel utah array\n manufacturer: BlackRock Microsystems\n\n location: Caudal, dorsal Pre-motor cortex, Left hemisphere\n",1,0.0,1.0
2,NaN,NaN,NaN,-1.0,"Pre-Motor Cortex, dorsal",1000Hz,"1 pynwb.ecephys.ElectrodeGroup at 0x4999977872\nFields:\n description: array corresponding to device implanted at PMd\n device: Utah Array(PMd) pynwb.device.Device at 0x5000072848\nFields:\n description: 96 channel utah array\n manufacturer: BlackRock Microsystems\n\n location: Caudal, dorsal Pre-motor cortex, Left hemisphere\n",1,0.0,1.0
3,NaN,NaN,NaN,-1.0,"Pre-Motor Cortex, dorsal",1000Hz,"1 pynwb.ecephys.ElectrodeGroup at 0x4999977872\nFields:\n description: array corresponding to device implanted at PMd\n device: Utah Array(PMd) pynwb.device.Device at 0x5000072848\nFields:\n description: 96 channel utah array\n manufacturer: BlackRock Microsystems\n\n location: Caudal, dorsal Pre-motor cortex, Left hemisphere\n",1,0.0,1.0


In [5]:
trialtable = nwb.trials.to_dataframe()
neurons = nwb.units.to_dataframe()['spike_times'] # Neuron id x Spike Time
time = np.round(nwb.processing['behavior'].data_interfaces['Position'].spatial_series['Hand'].timestamps[:]+ 0.0001,decimals=3)
hand = nwb.processing['behavior'].data_interfaces['Position'].spatial_series['Hand'].data[:] # x, y 
cursor = nwb.processing['behavior'].data_interfaces['Position'].spatial_series['Cursor'].data[:] # x, y
gocue = np.round(trialtable['go_cue_time'],decimals=3)
startmov = np.round(trialtable['move_begins_time'],decimals=3)
endmov = np.round(trialtable['move_ends_time'],decimals=3)
reactiontime = np.round(trialtable['reaction_time'],decimals=3)
target_pos = trialtable['target_positions']
num_neurons = neurons.shape[0]

In [6]:
num_trials1=0
num_trials2=len(trialtable)

In [7]:
data_list = {'firing_rates': [], 'events': [],'hand': [],'cursor': [],'result': [], 'maze_num_targets': [], 'maze_num_barriers': [], 'target_positions':[]}
for trial_id in tqdm(range(num_trials1, num_trials2)):

            min_T = np.round(trialtable['start_time'][trial_id],decimals=3) # Trial start time
            max_T = np.round(trialtable['stop_time'][trial_id],decimals=3) # Trial end time

            mi = np.nonzero(time>min_T)[0][0]
            ma = np.nonzero(time<max_T)[0][-1]

            grid = np.arange(min_T, max_T + 0.001, 0.001) #seconds
            grid = np.round(grid,decimals=3)
            grids = grid[:-1]
            gride = grid[1:]
            num_bins = len(grid)

            neurons_binned = np.zeros((num_bins, num_neurons))
            hand_binned = np.zeros((num_bins, 2))
            cursor_binned = np.zeros((num_bins, 2))
            targets_binned = np.zeros((num_bins,))
            events_binned = np.zeros((num_bins,))
            result= np.zeros((num_bins,))
            maze_barriers = np.zeros((num_bins,))
            maze_targets = np.zeros((num_bins,))
            reaction_t = np.zeros((num_bins,))
            targets_pos = np.zeros((num_bins,2))

            #targets_dir += target_dir_degree #target dir

            _, hand_binned[:,0] = np.histogram(hand[mi:ma,0], bins=num_bins-1)
            _, hand_binned[:,1] = np.histogram(hand[mi:ma,1], bins=num_bins-1)

            _, cursor_binned[:,1] = np.histogram(cursor[mi:ma,1], bins=num_bins-1)
            _, cursor_binned[:,1] = np.histogram(cursor[mi:ma,1], bins=num_bins-1)

            #hand_binned = hand[mi:ma]
            #cursor_binned = cursor[mi:ma]
            maze_barriers  += trialtable['maze_num_barriers'][trial_id]
            targets_pos += target_pos[trial_id][0]
            reaction_t += reactiontime[trial_id]

            maze_targets  += trialtable['maze_num_targets'][trial_id]

            result += trialtable['task_success'][trial_id]


            for n in range(num_neurons):
                # Rounding the spiketimes to two decimal places
                rounded_spike_times = np.round(neurons[n], decimals=3)

                neurons_binned[:, n] = np.isin(grid, rounded_spike_times).astype(int)

            if gocue[trial_id] > 0 and gocue[trial_id] <= grid[-1]:

                idx_go = np.where(grid == gocue[trial_id])[0][0]
                events_binned[idx_go] = 2 #go cue

            if startmov[trial_id] > 0:

                idx_startmov = np.where(grid == startmov[trial_id])[0][0]
                events_binned[idx_startmov] = 3 #move start

            if endmov[trial_id] > 0:
                idx_endmov = np.where(grid == endmov[trial_id])[0][0]
                events_binned[idx_endmov] = 4 #move end

            data_list['firing_rates'].append(neurons_binned)
            data_list['cursor'].append(cursor_binned)
            data_list['hand'].append(hand_binned)
            data_list['events'].append(events_binned)
            data_list['result'].append(result)
            data_list['maze_num_barriers'].append(maze_barriers)
            data_list['maze_num_targets'].append(maze_targets)
            data_list['target_positions'].append(targets_pos)

100%|██████████| 4044/4044 [3:01:22<00:00,  2.69s/it]  


In [8]:
data_list.keys()

dict_keys(['firing_rates', 'events', 'hand', 'cursor', 'result', 'maze_num_targets', 'maze_num_barriers', 'target_positions'])

In [9]:
fr = pd.DataFrame(np.vstack(data_list["firing_rates"]), columns=["Neuron"+str(i) for i in range(1, num_neurons+1)])
fr['trial_id'] = np.repeat(range(num_trials1, num_trials2), [arr.shape[0] for arr in data_list['firing_rates']])

In [10]:
fr

,Neuron1,Neuron2,Neuron3,Neuron4,Neuron5,Neuron6,Neuron7,Neuron8,Neuron9,Neuron10,...,Neuron184,Neuron185,Neuron186,Neuron187,Neuron188,Neuron189,Neuron190,Neuron191,Neuron192,trial_id
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0
2,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13109508,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4043
13109509,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4043
13109510,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4043
13109511,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4043


In [11]:
h = pd.DataFrame(np.vstack(data_list["hand"]),columns = ["hand_pos_x", "hand_pos_y"])
c = pd.DataFrame(np.vstack(data_list["cursor"]),columns = ["cursor_pos_x", "cursor_pos_y"])
tpos = pd.DataFrame(np.vstack(data_list["target_positions"]),columns = ["target_pos_x", "target_pos_y"])

In [12]:
tpos

,target_pos_x,target_pos_y
0,-119.0,-38.0
1,-119.0,-38.0
2,-119.0,-38.0
3,-119.0,-38.0
4,-119.0,-38.0
...,...,...
13109508,-107.0,78.0
13109509,-107.0,78.0
13109510,-107.0,78.0
13109511,-107.0,78.0


In [13]:
c

,cursor_pos_x,cursor_pos_y
0,0.0,-4.055316
1,0.0,-4.039651
2,0.0,-4.023986
3,0.0,-4.008321
4,0.0,-3.992656
...,...,...
13109508,0.0,145.501374
13109509,0.0,145.602386
13109510,0.0,145.703399
13109511,0.0,145.804411


In [14]:
h

,hand_pos_x,hand_pos_y
0,-135.681006,-153.006712
1,-135.623850,-152.956384
2,-135.566694,-152.906056
3,-135.509537,-152.855728
4,-135.452381,-152.805401
...,...,...
13109508,158.304320,83.513562
13109509,158.408669,83.614984
13109510,158.513018,83.716406
13109511,158.617367,83.817828


In [15]:
numtargets = pd.DataFrame(np.hstack(data_list["maze_num_targets"]), columns=['maze_num_target'])
numb = pd.DataFrame(np.hstack(data_list["maze_num_barriers"]), columns=['maze_num_barriers'])

In [16]:
numtargets

,maze_num_target
0,2.0
1,2.0
2,2.0
3,2.0
4,2.0
...,...
13109508,2.0
13109509,2.0
13109510,2.0
13109511,2.0


In [17]:
numb

,maze_num_barriers
0,4.0
1,4.0
2,4.0
3,4.0
4,4.0
...,...
13109508,4.0
13109509,4.0
13109510,4.0
13109511,4.0


In [18]:
event = pd.DataFrame(np.hstack(data_list["events"]), columns=['events'])

In [19]:
event

,events
0,0.0
1,0.0
2,0.0
3,0.0
4,0.0
...,...
13109508,0.0
13109509,0.0
13109510,0.0
13109511,0.0


In [20]:
r = pd.DataFrame(np.hstack(data_list["result"]), columns=['result'])

In [21]:
r

,result
0,0.0
1,0.0
2,0.0
3,0.0
4,0.0
...,...
13109508,1.0
13109509,1.0
13109510,1.0
13109511,1.0


In [22]:
ev = pd.concat([h,c,tpos,numtargets,numb,event,r], axis=1)

In [23]:
result = pd.concat([fr,ev], axis=1)

In [24]:
result

,Neuron1,Neuron2,Neuron3,Neuron4,Neuron5,Neuron6,Neuron7,Neuron8,Neuron9,Neuron10,...,hand_pos_x,hand_pos_y,cursor_pos_x,cursor_pos_y,target_pos_x,target_pos_y,maze_num_target,maze_num_barriers,events,result
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,...,-135.681006,-153.006712,0.0,-4.055316,-119.0,-38.0,2.0,4.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,-135.623850,-152.956384,0.0,-4.039651,-119.0,-38.0,2.0,4.0,0.0,0.0
2,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,-135.566694,-152.906056,0.0,-4.023986,-119.0,-38.0,2.0,4.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,-135.509537,-152.855728,0.0,-4.008321,-119.0,-38.0,2.0,4.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,-135.452381,-152.805401,0.0,-3.992656,-119.0,-38.0,2.0,4.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13109508,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,158.304320,83.513562,0.0,145.501374,-107.0,78.0,2.0,4.0,0.0,1.0
13109509,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,158.408669,83.614984,0.0,145.602386,-107.0,78.0,2.0,4.0,0.0,1.0
13109510,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,158.513018,83.716406,0.0,145.703399,-107.0,78.0,2.0,4.0,0.0,1.0
13109511,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,158.617367,83.817828,0.0,145.804411,-107.0,78.0,2.0,4.0,0.0,1.0


In [25]:
fr['Neuron1'].describe()

count    1.310951e+07
mean     1.146114e-03
std      3.383490e-02
min      0.000000e+00
25%      0.000000e+00
50%      0.000000e+00
75%      0.000000e+00
max      1.000000e+00
Name: Neuron1, dtype: float64

In [26]:
event['events'].value_counts()

events
0.0    13098950
2.0        4024
3.0        3607
4.0        2932
Name: count, dtype: int64

In [27]:
result

,Neuron1,Neuron2,Neuron3,Neuron4,Neuron5,Neuron6,Neuron7,Neuron8,Neuron9,Neuron10,...,hand_pos_x,hand_pos_y,cursor_pos_x,cursor_pos_y,target_pos_x,target_pos_y,maze_num_target,maze_num_barriers,events,result
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,...,-135.681006,-153.006712,0.0,-4.055316,-119.0,-38.0,2.0,4.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,-135.623850,-152.956384,0.0,-4.039651,-119.0,-38.0,2.0,4.0,0.0,0.0
2,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,-135.566694,-152.906056,0.0,-4.023986,-119.0,-38.0,2.0,4.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,-135.509537,-152.855728,0.0,-4.008321,-119.0,-38.0,2.0,4.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,-135.452381,-152.805401,0.0,-3.992656,-119.0,-38.0,2.0,4.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13109508,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,158.304320,83.513562,0.0,145.501374,-107.0,78.0,2.0,4.0,0.0,1.0
13109509,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,158.408669,83.614984,0.0,145.602386,-107.0,78.0,2.0,4.0,0.0,1.0
13109510,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,158.513018,83.716406,0.0,145.703399,-107.0,78.0,2.0,4.0,0.0,1.0
13109511,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,158.617367,83.817828,0.0,145.804411,-107.0,78.0,2.0,4.0,0.0,1.0


In [28]:
# Save the DataFrame to a Parquet file
parquet_file_path = 'processed_data/Churchland/sub-Nitschke_ses-20090920_behavior+ecephys.parquet' 
table = pa.Table.from_pandas(result)
pq.write_table(table, parquet_file_path)